#Подготовка
##Подключаем mpi4py ipyparallel

In [ ]:
%cd /content
!rm -rf *

In [ ]:
!pip3 install mpi4py
!pip3 install ipyparallel

## Cоздаем директории для расчетов

In [99]:
import os
results = r'/content/results'

list_names = ['SGD','Adam','Adagrad','RMSprop','Adadelta']
if not os.path.exists(results):
    os.makedirs(results)
for name in list_names:
  optim_dir = f'{results}/{name}'
  if not os.path.exists(optim_dir):
    os.makedirs(optim_dir)

## Скачиваем с [github](https://github.com/B3aRrrr/Parallel-and-distributed-computing-) репозиторий

In [98]:
!git clone https://github.com/B3aRrrr/Parallel-and-distributed-computing-  # clone
%cp -r /content/Parallel-and-distributed-computing-/"Colab Mpi+PyTorch utils"/* /content/.

Cloning into 'Parallel-and-distributed-computing-'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 47 (delta 19), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (47/47), 1.69 MiB | 3.64 MiB/s, done.


#Training parallel

In [ ]:
# %localhost slots=4
! mpirun -n 6 --allow-run-as-root --oversubscribe python3  mpiTrain.py

Optimizer: Adadelta - Start
Optimizer: RMSprop - Start
Optimizer: Adam - Start
Optimizer: Adagrad - Start
Optimizer: SGD - Start

Test set: Avg. loss: 2.3013, Accuracy: 1682/10000 (17%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.294372

Test set: Avg. loss: 2.2998, Accuracy: 1205/10000 (12%)


Test set: Avg. loss: 2.3061, Accuracy: 954/10000 (10%)


Test set: Avg. loss: 2.3090, Accuracy: 942/10000 (9%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.295907
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.318366
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.287235

Test set: Avg. loss: 2.3131, Accuracy: 917/10000 (9%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.318739
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.347028
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.258039
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.072896
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.224051
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.329914
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.299022
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.283331
Train Epoch: 1 [1280/60000 (2%

# Visualizations

And that's it. With just 3 epochs of training we already managed to achieve 97% accuracy on the test set! We started out with randomly initialized parameters and as expected only got about 10% accuracy on the test set before starting the training.

Let's plot our training curve.

In [92]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as offline

import numpy as np

titles_names = [f'Plot {optimizer}' for optimizer in list_names]
# len(list_names), 1
fig = make_subplots(
    rows=len(list_names), cols=1,
    subplot_titles=titles_names)

for i in range(4-1):
    name = list_names[i]
    with open(f'/content/results/{name}/train_losses.txt', 'r') as file:
        train_losses = list(map(float,file.read().replace('\n', '').replace('[','').replace(']','').split(sep=', ')))
    
    with open(f'/content/results/{name}/train_counter.txt', 'r') as file:
        train_counter = list(map(float,file.read().replace('\n', '').replace('[','').replace(']','').split(sep=', ')))

    with open(f'/content/results/{name}/test_losses.txt', 'r') as file:
        test_losses = list(map(float,file.read().replace('\n', '').replace('[','').replace(']','').split(sep=', ')))
    
    with open(f'/content/results/{name}/test_counter.txt', 'r') as file:
        test_counter = list(map(float,file.read().replace('\n', '').replace('[','').replace(']','').split(sep=', '))) 

    
    fig.add_trace(go.Scatter(x=train_counter,y=train_losses),               
                  row=i+1, col=1)
    
    fig.add_scatter(
        x=test_counter,
        y=test_losses, 
        mode="markers",                
        marker=dict(size=20,color="LightSeaGreen"),                
        #name="a", 
        row=i+1, col=1)

fig.update_layout(height=500, width=700,
                  title_text="Optimizers")

fig.show()